# <B> Evaluation </B>
* Container: codna_pytorch_py39

## AutoReload

In [54]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 1. parameter store 설정

In [55]:
import boto3
from utils.ssm import parameter_store

In [56]:
strRegionName=boto3.Session().region_name
pm = parameter_store(strRegionName)
prefix = pm.get_params(key="PREFIX")

## 2.Processing-job for evaluation

In [57]:
import os
import sagemaker
from sagemaker.pytorch.estimator import PyTorch
from sagemaker.workflow.execution_variables import ExecutionVariables
from sagemaker.processing import ProcessingInput, ProcessingOutput, FrameworkProcessor

* **Set Up SageMaker Experiment**
    - Create or load [SageMaker Experiment](https://docs.aws.amazon.com/sagemaker/latest/dg/experiments.html) for the example training job. This will create an experiment trial object in SageMaker.

* params for processing job

In [58]:
from time import strftime
from smexperiments.trial import Trial
from smexperiments.experiment import Experiment

In [59]:
def create_experiment(experiment_name):
    try:
        sm_experiment = Experiment.load(experiment_name)
    except:
        sm_experiment = Experiment.create(experiment_name=experiment_name)

In [60]:
def create_trial(experiment_name):
    create_date = strftime("%m%d-%H%M%s")
    sm_trial = Trial.create(trial_name=f'{experiment_name}-{create_date}',
                            experiment_name=experiment_name)

    job_name = f'{sm_trial.trial_name}'
    return job_name

In [61]:
local_mode = True

if local_mode: 
    instance_type = 'local_gpu'
    
    import os
    from sagemaker.local import LocalSession
    from nemo.utils.notebook_utils import download_an4
    
    sagemaker_session = LocalSession()
    sagemaker_session.config = {'local': {'local_code': True}}
    
else:
    instance_type = "ml.g4dn.xlarge"
    sagemaker_session = sagemaker.Session()
    wav_s3_uri = os.path.join(preprocessing_data_path, "an4", "wav")

sagemaker_role_arn = pm.get_params(key=prefix + '-SAGEMAKER-ROLE-ARN')
preprocessing_data_path = pm.get_params(key=prefix + '-PREP-DATA-PATH')
bucket_name = pm.get_params(key=prefix + '-BUCKET')
eval_image_uri = pm.get_params(key=''.join([prefix, '-IMAGE-URI']))
model_artifact_s3_uri = pm.get_params(key=prefix + '-MODEL-PATH').rsplit("/", 1)[0]
model_artifact_s3_uri = pm.get_params(key=prefix + '-MODEL-PATH')
test_manifest_s3_path = os.path.join(preprocessing_data_path, "an4", "test_manifest.json")
wav_s3_uri = os.path.join(preprocessing_data_path, "an4", "wav")
eval_output_s3_uri = os.path.join("s3://", bucket_name, "evaluation", "output", "eval_output")

code_location= os.path.join(
    "s3://{}".format(bucket_name),
    prefix,
    "evaluation",
    "backup_codes"
)

print (f"instance-type: {instance_type}")
print (f"image-uri: {eval_image_uri}")
print (f"role: {sagemaker_role_arn}")
print (f"bucket: {bucket_name}")
print (f"preprocessing-data-path: {preprocessing_data_path}")
print (f"model_artifact_s3_uri: {model_artifact_s3_uri}")
print (f"test_manifest_s3_path: {test_manifest_s3_path}")
print (f"wav_s3_uri: {wav_s3_uri}")
print (f"eval_output_s3_uri: {eval_output_s3_uri}")
print (f"sagemaker_session: {sagemaker_session}")

instance-type: local_gpu
image-uri: 419974056037.dkr.ecr.ap-northeast-2.amazonaws.com/nemo-test-training
role: arn:aws:iam::419974056037:role/service-role/AmazonSageMaker-ExecutionRole-20221206T163436
bucket: sm-nemo-bucket
preprocessing-data-path: s3://sm-nemo-bucket/nemo-asr/preprocessing/data
model_artifact_s3_uri: s3://sm-nemo-bucket/nemo-asr/training/model-output/nemo-asr-nemo-experiments-0321-04351679373354/model.tar.gz
test_manifest_s3_path: s3://sm-nemo-bucket/nemo-asr/preprocessing/data/an4/test_manifest.json
wav_s3_uri: s3://sm-nemo-bucket/nemo-asr/preprocessing/data/an4/wav
eval_output_s3_uri: s3://sm-nemo-bucket/evaluation/output/eval_output
sagemaker_session: <sagemaker.local.local_session.LocalSession object at 0x7fe928c8fe50>


* Define processing job

In [62]:
eval_processor = FrameworkProcessor(
    estimator_cls=PyTorch,
    framework_version="1.13.1",
    role=sagemaker_role_arn, 
    image_uri=eval_image_uri,
    instance_count=1,
    instance_type=instance_type,
    code_location=code_location,
    env={
        'MANIFEST_PATH': '/opt/ml/input/data/testing/an4/wav', 
        'WAV_PATH' : '/opt/ml/processing/input/wav'
        }
    )

In [63]:
experiment_name = pm.get_params(key=prefix + "-EXPERI-NAME")
create_experiment(experiment_name)
job_name = create_trial(experiment_name)

eval_processor.run(
    code="evaluate.py",
    source_dir="./code",
    inputs=[
        ProcessingInput(
            source=model_artifact_s3_uri,
            input_name="model_artifact",
            destination="/opt/ml/processing/model"
        ),
        ProcessingInput(
            source=test_manifest_s3_path,
            input_name="test_manifest_file",
            destination="/opt/ml/processing/input/manifest"
        ),
        ProcessingInput(
            source=wav_s3_uri,
            input_name="wav_dataset",
            destination="/opt/ml/processing/input/wav"
        ),
    ],
    outputs=[
        ProcessingOutput(
            output_name="eval-output-data",
            source="/opt/ml/processing/evaluation",
            destination=eval_output_s3_uri
            
        ),
    ],
    job_name=job_name,
    experiment_config={
      'TrialName': job_name,
      'TrialComponentDisplayName': job_name,
    },
    wait=True
)

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:sagemaker.processing:Uploaded ./code to s3://sm-nemo-bucket/nemo-asr/evaluation/backup_codes/nemo-asr-nemo-experiments-0321-04381679373482/source/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://sm-nemo-bucket/nemo-asr/evaluation/backup_codes/nemo-asr-nemo-experiments-0321-04381679373482/source/runproc.sh
INFO:sagemaker:Creating processing-job with name nemo-asr-nemo-experiments-0321-04381679373482
INFO:sagemaker.local.local_session:Starting processing job
INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:sagemaker.local.image:No AWS credentials found in session but credentials from EC2 Metadata Service are available.
INFO:sagemak

Creating 15z3844ex7-algo-1-5av77 ... 
Creating 15z3844ex7-algo-1-5av77 ... done
Attaching to 15z3844ex7-algo-1-5av77
15z3844ex7-algo-1-5av77 | [NeMo W 2023-03-21 04:38:50 optimizers:66] Could not import distributed_fused_adam optimizer from Apex
15z3844ex7-algo-1-5av77 | [NeMo W 2023-03-21 04:38:52 experimental:27] Module <class 'nemo.collections.asr.modules.audio_modules.SpectrogramToMultichannelFeatures'> is experimental, not ready for production and is not fully supported. Use at your own risk.
15z3844ex7-algo-1-5av77 | Loading nemo model.
15z3844ex7-algo-1-5av77 | checkpoint_path : trained_model/Conformer-CTC-Char1/2023-03-21_04-36-34/checkpoints/Conformer-CTC-Char1--val_wer=1.0000-epoch=2-last.ckpt
15z3844ex7-algo-1-5av77 | [NeMo W 2023-03-21 04:38:57 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
15z3844ex7-algo-1-5av77 |     Train config : 
15z38

In [64]:
eval_processor

In [65]:
pm.put_params(key="-".join([prefix, "EVAL-OUTPUT-PATH"]), value=eval_output_s3_uri+"evaluation.json", overwrite=True)

'Store suceess'

In [66]:
pm.get_params(key="-".join([prefix, "EVAL-OUTPUT-PATH"]))

's3://sm-nemo-bucket/evaluation/output/eval_outputevaluation.json'

In [67]:
pm.get_params(key="-".join([prefix, "BUCKET"]))

'sm-nemo-bucket'